In [1]:
# Copied from HeroesOfPymoli_starter.ipynb
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# Player Count -- Display total number of players
uniquePlayers = purchase_data.drop_duplicates(subset=["SN"], keep="first")
playerCount = uniquePlayers["SN"].count()
summaryCountDf = pd.DataFrame({"Total Players": [playerCount]})
summaryCountDf

,Total Players
0,576


In [3]:
# Purchasing Analysis -- Num unique items, avg purchace price, total num of purchases, total revenue

uniqueItems = len(purchase_data["Item Name"].unique())
avgPurchase = purchase_data["Price"].mean()
avgPurchaseRound = round(avgPurchase,2)
totPurchase = purchase_data["Purchase ID"].count()
totRevenue = purchase_data["Price"].sum()


summaryPurchAnalysisDf = pd.DataFrame({"Number of Unique Items": [uniqueItems],
                                   "Average Price": [avgPurchaseRound],
                                   "Number of Purchases": [totPurchase],
                                   "Total Revenue": [totRevenue]})
summaryPurchAnalysisDf["Average Price"] = summaryPurchAnalysisDf["Average Price"].astype(float).map("${:,.2f}".format)
summaryPurchAnalysisDf["Total Revenue"] = summaryPurchAnalysisDf["Total Revenue"].astype(float).map("${:,.2f}".format)
summaryPurchAnalysisDf

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
# Gender Demopgraphics -- Perc and Count of each gender (Male, Female, Other)
summaryGenderDF = pd.DataFrame(index=["Male","Female", "Other / Non-Disclosed"], columns=["Total Count", "Percentage of Players"])

genderTotal = uniquePlayers["Gender"].value_counts()

percentTotal = uniquePlayers["Gender"].value_counts(normalize=True)*100
percentTotalRound = round(percentTotal,2)

totalMale = genderTotal[0]
percentMale = percentTotalRound[0]
summaryGenderDF.loc["Male"] = [totalMale, percentMale]

totalFemale = genderTotal[1]
percentFemale = percentTotalRound[1]
summaryGenderDF.loc["Female"] = [totalFemale, percentFemale]

totalOther = genderTotal[2]
percentOther = percentTotalRound[2]
summaryGenderDF.loc["Other / Non-Disclosed"] = [totalOther, percentOther]


summaryGenderDF

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [5]:
# Purchasing Analysis (Gender) -- For each gender purchase count, avg perchase price, 
# total purchase value, avg total purchase by person

summaryGenderAnalysisDf = pd.DataFrame(index=["Male","Female", "Other / Non-Disclosed"], 
                                       columns=["Purchase Count", "Average Purchase Price", "Total Purchase Price",
                                               "Average Total Purchase per Person"])

GenderPurchaseCount = purchase_data.groupby(["Gender"]).Price.count()
GenderAvgPurchasePrice = purchase_data.groupby(["Gender"]).Price.mean()
GenderAvgPurchasePriceRounded = round(GenderAvgPurchasePrice,2)
GenderTotalPurchase = purchase_data.groupby(["Gender"]).Price.sum()
GenderAvgPerPerson = GenderTotalPurchase / genderTotal 


MalePurchaseCount = GenderPurchaseCount[1]
MaleAvgPurchasePrice = GenderAvgPurchasePriceRounded[1]
MaleTotalPurchasePrice = GenderTotalPurchase[1]
MaleAvgPerPerson = GenderAvgPerPerson[1]
summaryGenderAnalysisDf.loc["Male"] = [MalePurchaseCount, MaleAvgPurchasePrice, MaleTotalPurchasePrice, MaleAvgPerPerson]

FemalePurchaseCount = GenderPurchaseCount[0]
FemaleAvgPurchasePrice = GenderAvgPurchasePriceRounded[0]
FemaleTotalPurchasePrice = GenderTotalPurchase[0]
FemaleAvgPerPerson = GenderAvgPerPerson[0]
summaryGenderAnalysisDf.loc["Female"] = [FemalePurchaseCount, FemaleAvgPurchasePrice, 
                                         FemaleTotalPurchasePrice, FemaleAvgPerPerson]

OtherPurchaseCount = GenderPurchaseCount[2]
OtherAvgPurchasePrice = GenderAvgPurchasePriceRounded[2]
OtherTotalPurchasePrice = GenderTotalPurchase[2]
OtherAvgPerPerson = GenderAvgPerPerson[2]
summaryGenderAnalysisDf.loc["Other / Non-Disclosed"] = [OtherPurchaseCount, OtherAvgPurchasePrice,                                                        OtherTotalPurchasePrice, OtherAvgPerPerson]

summaryGenderAnalysisDf["Average Purchase Price"] = summaryGenderAnalysisDf["Average Purchase Price"].astype(float).map("${:,.2f}".format)
summaryGenderAnalysisDf["Total Purchase Price"] = summaryGenderAnalysisDf["Total Purchase Price"].astype(float).map("${:,.2f}".format)
summaryGenderAnalysisDf["Average Total Purchase per Person"] = summaryGenderAnalysisDf["Average Total Purchase per Person"].astype(float).map("${:,.2f}".format)

summaryGenderAnalysisDf

,Purchase Count,Average Purchase Price,Total Purchase Price,Average Total Purchase per Person
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
# Age Demographics -- bins of 4 years starting at 10, purchase count, avg price, total value, avg purchase total
groupLabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
bins = [1,9,14,19,24,29,34,39,100]
summaryAgeDemo = pd.DataFrame(index=groupLabels, 
                              columns=["Total Count", "Percentage of Players"])
purchase_data['Age Groups'] = pd.cut(purchase_data['Age'], bins, labels=groupLabels)
ageGroups = purchase_data.groupby('Age Groups').nunique()
summaryAgeDemo["Total Count"] = ageGroups['SN']
pecentAgeGroup = (ageGroups['SN'] / playerCount)
summaryAgeDemo["Percentage of Players"] = pecentAgeGroup
summaryAgeDemo["Percentage of Players"] = summaryAgeDemo["Percentage of Players"].astype(float).map("{:.2%}".format)

summaryAgeDemo

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [7]:
# Purchasing Analysis (Age)
summaryAgeAnalysis = pd.DataFrame(index=groupLabels, 
                                  columns=['Purchase Count', 'Average Purchase Price', 'Total Purchase Value',
                                           'Avg Total Purchase per Person'])
summaryAgeAnalysis['Purchase Count'] = ageGroups['Purchase ID']
summaryAgeAnalysis['Total Purchase Value'] = purchase_data.groupby('Age Groups')['Price'].sum().astype(float).map("${:,.2f}".format)
summaryAgeAnalysis['Average Purchase Price'] = (purchase_data.groupby('Age Groups')['Price'].mean()).astype(float).map("${:,.2f}".format)
summaryAgeAnalysis['Avg Total Purchase per Person'] = (purchase_data.groupby('Age Groups')['Price'].sum() / ageGroups['SN']).astype(float).map("${:,.2f}".format)

summaryAgeAnalysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [29]:
# 5 Highest Spenders
#groupedPurchaseNoDupes = uniquePlayers.groupby(by="SN")
topFiveSpenders = purchase_data
topFiveSpenders["Total Spent"] =  purchase_data.groupby('SN')['Price'].sum()
topFiveSpenders.groupby(["Gender"]).head()

purchase_data.group


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Groups,Total Spent
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24,NaN
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+,NaN
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24,NaN
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24,NaN
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24,NaN
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58,35-39,NaN
15,15,Lisassa64,21,Female,98,"Deadline, Voice Of Subtlety",2.89,20-24,NaN
18,18,Reunasu60,22,Female,82,Nirvana,4.90,20-24,NaN
22,22,Siarithria38,38,Other / Non-Disclosed,24,Warped Fetish,3.81,35-39,NaN
38,38,Reulae52,10,Female,116,Renewed Skeletal Katana,4.18,10-14,NaN


In [9]:
# Most Popular items -- List 5 most popular items by ID, Name, Purchase Count, Item Price, Total Value

In [10]:
# Most Profitable items -- List 5 most profitable items by ID, Name, Purchase Count, Item Price, Total Value